## Capacity Contribution Analysis - CAISO


Resource Adequacy (RA) -  refers to the part of planning that assesses whether a Power system has an appropriate set of resources to meet demand, consistent with demand reliability standard/ criterion[[Page - 47]](https://www.bchydro.com/content/dam/BCHydro/customer-portal/documents/corporate/regulatory-planning-documents/integrated-resource-plans/current-plan/2025-irp-tac-mtg4-20241101-presentation.pdf)

Resource adequacy studies are very important to systems with more weather-dependent power/ energy resources. CAISO esources are a mix of

* Fossil Steam
* Hydro
* Nuclear
* Renewable
* Storage
* Combustion Turbine

This study aims to analyse the CAISO resources available for the year 2023 contributing to capacity/ resource adequacy

Dataset obtained from [CPUC Website](https://files.cpuc.ca.gov/energy/modeling/2023PSP/SERVM_GeneratorList_20231005.xlsx)

In [1]:
import pandas as pd

In [2]:
import altair as alt

In [3]:
data_gen = pd.read_excel('SERVM_GeneratorList_20231005.xlsx', header=1, sheet_name='GenList20231005')

In [4]:
data_gen['insvdt'] = pd.to_datetime(data_gen['insvdt'], format='mixed')
data_gen['retdt'] = pd.to_datetime(data_gen['retdt'], format='%Y%m%d')

In [5]:
data_gen.replace({'Unit Type':{'F':'Fossil Steam',
                               'H':'Hydro',
                               'N':'Nuclear',
                               'R':'Renewable',
                               'S':'Storage',
                               'T':'Combustion Turbine',
                               'C':'Curtailable load'}}, inplace = True)

In [6]:
data_gen['Unit Type'].unique()

array(['Renewable', 'Storage', 'Combustion Turbine', 'Fossil Steam',
       'Curtailable load', 'Hydro', 'Nuclear'], dtype=object)

The dataset gives CAISO and Non-CAISO resources as well as resources that are online and not online as of 1/2023
* Online = CAISO unit that is online
* Not online as of 1/2023 = CAISO project that is not yet online as of 1/2023 regardless of insvdt value
* Terminated = indicates a project has been terminated
* Non-CAISO = unit primarily serves non-CAISO region"


In [7]:
data_gen['Flag for CAISO-contracted and either Not online as of 1/2023 or terminated as of 1/2023'].unique()

array(['Non-CAISO', 'Not online as of 1/2023', 'Online', 'Terminated'],
      dtype=object)


The projects that are terminated are excluded from further analysis


In [8]:
data_gen_EandC_all = data_gen[(data_gen['Flag for CAISO-contracted and either Not online as of 1/2023 or terminated as of 1/2023'] != 'Terminated')]

To find out the existing and committed resources available for the FY2023, the above dataset is filtered

In [9]:
data_gen_E_all = data_gen_EandC_all[(data_gen_EandC_all['insvdt'].dt.year < 2024) & (data_gen_EandC_all['retdt'].dt.year >= 2023)]

In [10]:
len(data_gen_E_all[data_gen_E_all['Flag for CAISO-contracted and either Not online as of 1/2023 or terminated as of 1/2023']=='Not online as of 1/2023'])

276

Out of the filtered resources, even though the in-service year is 2023, 276 resources are flagged as not online as of 1/2023. Hence these resources can not be considered with existing resources and needed to be filtered out.

In [11]:
data_gen_E_online = data_gen_E_all[(data_gen_E_all['Flag for CAISO-contracted and either Not online as of 1/2023 or terminated as of 1/2023'] != 'Not online as of 1/2023')]

In [12]:
data_gen_E_online[['Capmax']].sum()[0]

205075.54124233001


The total installed capacity that CPUC can rely on to meet demand is 205075.54 MW. These capacity resources are a mixture of Fossil Steam, Hydro, Nuclear, Renewable, Storage, and Combustion Turbine. The contribution of each of these resources are shown in the bar chart below. 



In [13]:
capacity_chart = alt.Chart(data_gen_E_online).mark_bar(height = 15, color='orange').encode(
alt.Y('Unit Type', title='', sort = 'x'),
alt.X('sum(Capmax)', title='Capacity')).properties(height = 200,width=500)

In [14]:
capacity_chart + capacity_chart.mark_text(align='left', dx = 3).encode(text=alt.Text('sum(Capmax)'))

alt.LayerChart(...)

Dependable generating capacity (non-variable resources) CPUC can rely can be from Fossil Steam, Combustion Turbine, Nuclear and Hydro resources.

In [15]:
list_dependable = ['Fossil Steam','Hydro','Nuclear','Combustion Turbine']

In [16]:
CPUC_dependable = data_gen_E_online[data_gen_E_online['Unit Type'].isin(list_dependable)]

In [17]:
CPUC_dependable[['Capmax']].sum()[0]

112743.8114132

CAISO Existing Resources 

In [18]:
data_gen_E_online_CAISO = data_gen_E_online[data_gen_E_online['Flag for CAISO-contracted and either Not online as of 1/2023 or terminated as of 1/2023'] == 'Online']

In [19]:
data_gen_E_online_CAISO[['Capmax']].sum()[0]

97677.82524233

Total installed capacity of the CAISO system is a combination of 
- Dependable capacity obtained from Fossil Steam, Combustion Turbine, Nuclear and Hydro resources
- Variable capacity from renewable resources 
- Capacity from storage 
- Capacity from Demand response program - Load Curtailment

As menitioned above the dependable capacity contribution for the CAISO system comes from Fossil Steam, Combustion Turbine, Nuclear and Hydro resources. 

In [20]:
CAISO_dependable = data_gen_E_online_CAISO[data_gen_E_online_CAISO['Unit Type'].isin(list_dependable)]

In [21]:
CAISO_dependable[['Capmax']].sum()[0]

38550.6254132


The dependable capacity of the CAISO system is 38550.63MW. These capacity resources are a mixture of Fossil Steam, Hydro, Nuclear and Combustion Turbine. The contribution of each of these resources are shown in the bar chart below. 

In [22]:
capacity_chart_CAISO = alt.Chart(CAISO_dependable).mark_bar(height = 15).encode(
alt.Y('Unit Type', title='', sort = 'x'),
alt.X('sum(Capmax)', title='Capacity')).properties(height = 200,width=500)

In [23]:
capacity_chart_CAISO + capacity_chart_CAISO.mark_text(align='left', dx = 3).encode(text=alt.Text('sum(Capmax)'))

alt.LayerChart(...)

In [24]:
CAISO_renewable = data_gen_E_online_CAISO[data_gen_E_online_CAISO['Unit Type']=='Renewable']

In [25]:
CAISO_renewable[['Capmax']].sum()[0]

50397.990000000005

Renewable energy resources in CASIO system consists primarily of Wind and Solar. The contribution of renewable resources to CAISO installed capacity is shown in the following bar chart. 

In [26]:
renewable_chart_CAISO = alt.Chart(CAISO_renewable).mark_bar(height = 15, color='lightblue').encode(
alt.Y('Unit Category', title='', sort = 'x'),
alt.X('sum(Capmax)', title='Capacity'),
tooltip = ['sum(Capmax)'] ).properties(height = 600,width=500)

In [27]:
renewable_chart_CAISO + renewable_chart_CAISO.mark_text(align='left').encode(text=alt.Text('sum(Capmax)'))

alt.LayerChart(...)

CAISO has categorised the demand side capacity contributions under the renewable resources. Hence from the chart the capacity contributions from EV, AAFS, BTMStorageShapeCharge, BTMStorgeShapeDischarge, TOU_LoadIncrease, TOU_LoadDecrease, AAEE can also be observed 



In [28]:
CAISO_storage = data_gen_E_online_CAISO[data_gen_E_online_CAISO['Unit Type']=='Storage']

In [29]:
CAISO_storage[['Capmax']].sum()[0]

6455.1900000000005

In [30]:
storage_chart_CAISO = alt.Chart(CAISO_storage).mark_bar(height = 15, color='teal').encode(
alt.Y('Unit Category', title='', sort = 'x'),
alt.X('sum(Capmax)', title='Capacity')).properties(height = 200,width=500)

In [31]:
storage_chart_CAISO + storage_chart_CAISO.mark_text(align='left', dx = 3).encode(text=alt.Text('sum(Capmax)'))

alt.LayerChart(...)


The curtailable load is a load that can be reduced or shut off temporarily at the request of CAISO to manage the demand response


In [32]:
CAISO_curtailable_load = data_gen_E_online_CAISO[data_gen_E_online_CAISO['Unit Type']=='Curtailable load']

In [33]:
CAISO_curtailable_load[['Capmax']].sum()[0]

2274.01982913

### Conclusion


This study was conducted to learn about various resources of CAISO system contributing to the system capacity. It was seen that the CAISO systems capacity contribution for the year 2023 was from various resources including renewables, non renewables, storage and DR programs. 

Total installed capacity of the CAISO system in 2023 is 97677.825 MW, with each type of resource capacity contribution as listed below
- Dependable capacity obtained from Fossil Steam, Combustion Turbine, Nuclear and Hydro resources - 38550.63 MW
- Variable capacity from renewable resources – 50397.99 MW
- Capacity from storage – 6455 MW
- Capacity from Demand response program - Load Curtailment – 2274.019 MW

